## Open3d ICP用例
Open3d是一个结构精简, 功能全面的3D数据处理库, 且也有相应的维护与管理. 尽管点云处理仅是其核心功能的一部分, 但是足够我们进行常规的处理需要了.

下面进行迭代最近点算法(Iterative Closest Point, ICP)的学习和使用.

----------------------
### 0. 粗配准 Global registration
[粗配准](http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html)

粗配准不需要初始变换矩阵 `trans_init`, 结果很粗放, 但是可以为进一步ICP配准提供*初值*.

下面使用了最简单的*随机采样一致RANSAC*配准算法, 可以得到**粗对齐矩阵**.

In [1]:
import open3d as o3d
import numpy as np
import copy
import time

def draw_registration_result(source, target, transformation):
    """
    可视化函数
    :param source:
    :param target:
    :param transformation:
    :return:
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4559,
                                      front=[0.6452, -0.3036, -0.7011],
                                      lookat=[1.9892, 2.0208, 1.8945],
                                      up=[-0.2779, -0.9482, 0.1556])


def preprocess_point_cloud(pcd, voxel_size):
    """
    点云预处理 降采样
    :param pcd: 点云数据
    :param voxel_size: 体素采样大小
    :return: 降采样后点云, FPFH点快速特征直方图
    """
    print(":: Downsample with a voxel size %.3f." % voxel_size)
    pcd_down = pcd.voxel_down_sample(voxel_size)

    radius_normal = voxel_size * 2
    print(":: Estimate normal with search radius %.3f." % radius_normal)
    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_normal, max_nn=30))

    radius_feature = voxel_size * 5
    print(":: Compute FPFH feature with search radius %.3f." % radius_feature)
    pcd_fpfh = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=radius_feature, max_nn=100))
    return pcd_down, pcd_fpfh


def prepare_dataset(voxel_size):
    """
    点云文件读取和整理函数
    :param voxel_size: 体素大小
    :return: source, target, source_down, target_down, source_fpfh, target_fpfh
    """
    print(":: Load two point clouds and disturb initial pose.")
    source = o3d.io.read_point_cloud("../data/cloud_bin_0.pcd")
    target = o3d.io.read_point_cloud("../data/cloud_bin_1.pcd")
    # trans_init = np.asarray([[0.0, 0.0, 1.0, 0.0], [1.0, 0.0, 0.0, 0.0],
    #                          [0.0, 1.0, 0.0, 0.0], [0.0, 0.0, 0.0, 1.0]])
    trans_init = np.identity(4)
    source.transform(trans_init)
    draw_registration_result(source, target, np.identity(4))

    source_down, source_fpfh = preprocess_point_cloud(source, voxel_size)
    target_down, target_fpfh = preprocess_point_cloud(target, voxel_size)
    return source, target, source_down, target_down, source_fpfh, target_fpfh

voxel_size = 0.05  # means 5cm for this dataset
source, target, source_down, target_down, source_fpfh, target_fpfh = prepare_dataset(
    voxel_size)

def execute_global_registration(source_down, target_down, source_fpfh,
                                target_fpfh, voxel_size):
    """
    全局匹配函数
    :param source_down:
    :param target_down:
    :param source_fpfh:
    :param target_fpfh:
    :param voxel_size:
    :return:
    """
    distance_threshold = voxel_size * 1.5
    print(":: RANSAC registration on downsampled point clouds.")
    print("   Since the downsampling voxel size is %.3f," % voxel_size)
    print("   we use a liberal distance threshold %.3f." % distance_threshold)
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh, True,
        distance_threshold,
        o3d.pipelines.registration.TransformationEstimationPointToPoint(False),
        3, [
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(
                0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                distance_threshold)
        ], o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999))
    return result


result_ransac = execute_global_registration(source_down, target_down,
                                            source_fpfh, target_fpfh,
                                            voxel_size)

trans_init = np.array(result_ransac.transformation)
print("全局配准结果:", result_ransac)
print("全局配准矩阵:", trans_init)

draw_registration_result(source_down, target_down, result_ransac.transformation)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
:: Load two point clouds and disturb initial pose.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: Downsample with a voxel size 0.050.
:: Estimate normal with search radius 0.100.
:: Compute FPFH feature with search radius 0.250.
:: RANSAC registration on downsampled point clouds.
   Since the downsampling voxel size is 0.050,
   we use a liberal distance threshold 0.075.
全局配准结果: RegistrationResult with fitness=4.025465e-01, inlier_rmse=4.076945e-02, and correspondence_set size of 411
Access transformation to get result.
全局配准矩阵: [[ 0.84353399  0.01539879 -0.536855    0.61036551]
 [-0.15436482  0.96436303 -0.21488475  0.82958226]
 [ 0.51441414  0.26413411  0.81585002 -1.45666251]
 [ 0.          0.          0.          1.        ]]


#### 0.1 快速全局匹配

In [2]:
def execute_fast_global_registration(source_down, target_down, source_fpfh,
                                     target_fpfh, voxel_size):
    distance_threshold = voxel_size * 0.5
    print(":: Apply fast global registration with distance threshold %.3f" \
            % distance_threshold)
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        source_down, target_down, source_fpfh, target_fpfh,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=distance_threshold))
    return result

start = time.time()
result_fast = execute_fast_global_registration(source_down, target_down,
                                               source_fpfh, target_fpfh,
                                               voxel_size)
print("Fast global registration took %.3f sec.\n" % (time.time() - start))
print(result_fast)
draw_registration_result(source_down, target_down, result_fast.transformation)

trans_init = np.array(result_fast.transformation)
print(result_fast.transformation)

:: Apply fast global registration with distance threshold 0.025
Fast global registration took 0.085 sec.

RegistrationResult with fitness=5.073529e-01, inlier_rmse=1.745625e-02, and correspondence_set size of 2415
Access transformation to get result.
[[ 0.8417196   0.00736148 -0.53986473  0.63798603]
 [-0.1486846   0.96440503 -0.2186683   0.81391545]
 [ 0.51903854  0.26432696  0.81285315 -1.4787301 ]
 [-0.          0.         -0.          1.        ]]


清空之前数据, 防止串扰

In [3]:
# %reset

--------------------------------------------------

### 1. 官方用例
主要参考: [ICP registration](http://www.open3d.org/docs/release/tutorial/pipelines/icp_registration.html)


In [4]:
import open3d as o3d
import numpy as np
import copy

def draw_registration_result(source, target, transformation):
    """
    点云配准可视化函数
    :param source: 待配准点云 黄色
    :param target: 目标点云 青绿色
    :param transformation: 变换矩阵
    :return: None
    """
    source_temp = copy.deepcopy(source)
    target_temp = copy.deepcopy(target)
    source_temp.paint_uniform_color([1, 0.706, 0])
    target_temp.paint_uniform_color([0, 0.651, 0.929])
    source_temp.transform(transformation)
    o3d.visualization.draw_geometries([source_temp, target_temp],
                                      zoom=0.4459,
                                      front=[0.9288, -0.2951, -0.2242],
                                      lookat=[1.6784, 2.0612, 1.4451],
                                      up=[-0.3402, -0.9189, -0.1996])


初始的对齐一般使用 `Global registration` 方法 [例程](http://www.open3d.org/docs/release/tutorial/pipelines/global_registration.html)


In [5]:
# 读取点云文件
source = o3d.io.read_point_cloud("../data/cloud_bin_0.pcd")
target = o3d.io.read_point_cloud("../data/cloud_bin_1.pcd")
threshold = 0.02    # 匹配阈值

# Global registration 矩阵

# trans_init = np.identity(4)
# 教程给出的init
# trans_init = np.asarray([[0.862, 0.011, -0.507, 0.5],
#                          [-0.139, 0.967, -0.215, 0.7],
#                          [0.487, 0.255, 0.835, -1.4], [0.0, 0.0, 0.0, 1.0]])

draw_registration_result(source, target, trans_init)

In [6]:
# 第一次 匹配
print("1 alignment")
evaluation = o3d.pipelines.registration.evaluate_registration(
    source, target, threshold, trans_init)
print(evaluation)

1 alignment
RegistrationResult with fitness=6.209068e-01, inlier_rmse=7.601182e-03, and correspondence_set size of 123458
Access transformation to get result.


#### 点对点比较



In [7]:
import time

# 开始时间
startTime = time.time()

# ICP 配准 默认30次迭代
print("Apply point-to-point ICP")
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint())
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
endTime = time.time()
print('运算时间: {} s'.format(endTime - startTime))

Apply point-to-point ICP
RegistrationResult with fitness=6.211582e-01, inlier_rmse=6.564378e-03, and correspondence_set size of 123508
Access transformation to get result.
Transformation is:
[[ 0.84048718  0.00732684 -0.54178188  0.64405486]
 [-0.14786229  0.96505781 -0.21633345  0.81039089]
 [ 0.5212658   0.26193461  0.81220209 -1.48424266]
 [ 0.          0.          0.          1.        ]]
运算时间: 1.3550660610198975 s


In [8]:
draw_registration_result(source, target, reg_p2p.transformation)


增加迭代次数, `max_iteration` 2000次.
匹配效果有了较好提升, 训练时间也相应变长了不少.


In [9]:
# 开始时间
startTime = time.time()

# ICP 配准
reg_p2p = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPoint(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2p)
print("Transformation is:")
print(reg_p2p.transformation)
endTime = time.time()
print('运算时间: {} s'.format(endTime - startTime))
# draw_registration_result(source, target, reg_p2p.transformation)


RegistrationResult with fitness=6.211582e-01, inlier_rmse=6.564378e-03, and correspondence_set size of 123508
Access transformation to get result.
Transformation is:
[[ 0.84048718  0.00732684 -0.54178188  0.64405486]
 [-0.14786229  0.96505781 -0.21633345  0.81039089]
 [ 0.5212658   0.26193461  0.81220209 -1.48424266]
 [ 0.          0.          0.          1.        ]]
运算时间: 1.8823580741882324 s


#### 点对面比较
点对面比较效果很好, 且运算速度很快.

In [10]:
# 开始时间
startTime = time.time()

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane())
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
endTime = time.time()
print('运算时间: {} s'.format(endTime - startTime))

# draw_registration_result(source, target, reg_p2l.transformation)


Apply point-to-plane ICP
RegistrationResult with fitness=6.210275e-01, inlier_rmse=6.565173e-03, and correspondence_set size of 123482
Access transformation to get result.
Transformation is:
[[ 0.84047184  0.00663112 -0.54181464  0.64487082]
 [-0.14734413  0.9650432  -0.21675177  0.8095104 ]
 [ 0.52143723  0.26200696  0.8120687  -1.48472365]
 [ 0.          0.          0.          1.        ]]
运算时间: 0.6929140090942383 s


增加迭代次数, `max_iteration` 2000次.
相比30次, 变化不大, 计算时长也近似.

> 注意到如果使用很好的初始变换矩阵, 运算时间将会大大缩短.

In [11]:
# 开始时间
startTime = time.time()

print("Apply point-to-plane ICP")
reg_p2l = o3d.pipelines.registration.registration_icp(
    source, target, threshold, trans_init,
    o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=2000))
print(reg_p2l)
print("Transformation is:")
print(reg_p2l.transformation)
endTime = time.time()
print('运算时间: {} s'.format(endTime - startTime))

draw_registration_result(source, target, reg_p2l.transformation)


Apply point-to-plane ICP
RegistrationResult with fitness=6.210275e-01, inlier_rmse=6.565173e-03, and correspondence_set size of 123482
Access transformation to get result.
Transformation is:
[[ 0.84047184  0.00663112 -0.54181464  0.64487082]
 [-0.14734413  0.9650432  -0.21675177  0.8095104 ]
 [ 0.52143723  0.26200696  0.8120687  -1.48472365]
 [ 0.          0.          0.          1.        ]]
运算时间: 0.794975757598877 s


In [12]:
# 点云转 ndarray
import numpy as np

aaa = np.asarray(source_down.points)
print(aaa)





[[1.29708717 2.37313138 2.24693085]
 [1.30058623 2.38606423 2.15585233]
 [1.39567678 2.37072615 2.20223917]
 ...
 [2.72785974 2.34765625 1.19140625]
 [1.37036602 2.41163762 0.84374872]
 [1.29958513 2.39776017 1.15736005]]
